In [1]:
// put the spark install from the base notebook image onto Ammonite's classpath
java.nio.file.Files.list(java.nio.file.Paths.get("/opt/spark/jars")).toArray.map(_.toString).foreach { fname =>
  val path = java.nio.file.FileSystems.getDefault().getPath(fname)
  val x = ammonite.ops.Path(path)
  interp.load.cp(x)
}
// Load the ammonite-spark package to get AmmoniteSparkSession
import $ivy.`sh.almond::ammonite-spark:0.1.1`

import $ivy.$                                

In [2]:
import $ivy.`org.apache.spark::spark-sql-kafka-0-10:2.2.0`

import $ivy.$                                             

In [3]:
//import $ivy.`org.isarnproject::isarn-sketches-spark:0.3.1-sp2.2-py2.7`
import $ivy.`org.isarnproject::isarn-sketches-spark:0.3.1-topk-1-sp2.2-py2.7`

import $ivy.$                                                                

In [4]:
import org.apache.spark.sql._
val spark = {
    AmmoniteSparkSession.builder()
      .master("local[2]")
      .getOrCreate()
  }

Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
18/09/27 17:57:52 INFO SparkContext: Running Spark version 2.2.0
18/09/27 17:57:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
18/09/27 17:57:52 INFO SparkContext: Submitted application: 28da45ca-7f94-464c-a632-6e2985e4b08f
18/09/27 17:57:52 INFO SecurityManager: Changing view acls to: 1000130000
18/09/27 17:57:52 INFO SecurityManager: Changing modify acls to: 1000130000
18/09/27 17:57:52 INFO SecurityManager: Changing view acls groups to: 
18/09/27 17:57:52 INFO SecurityManager: Changing modify acls groups to: 
18/09/27 17:57:52 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(1000130000); groups with view permissions: Set(); users  with modify permissions: Set(1000130000); groups with modify permissions: Set()
18/09/27 17:57:53 INFO Utils: Successfully started

18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-xml_2.11/1.1.0/scala-xml_2.11-1.1.0.jar at spark://172.17.0.10:35101/jars/scala-xml_2.11-1.1.0.jar with timestamp 1538071073638
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/.cache/coursier/v1/https/repo1.maven.org/maven2/org/jline/jline-terminal-jna/3.6.2/jline-terminal-jna-3.6.2.jar at spark://172.17.0.10:35101/jars/jline-terminal-jna-3.6.2.jar with timestamp 1538071073638
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/ujson_2.11/0.6.6/ujson_2.11-0.6.6.jar at spark://172.17.0.10:35101/jars/ujson_2.11-0.6.6.jar with timestamp 1538071073639
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/.cache/coursier/v1/https/repo1.maven.org/maven2/com/github/javaparser/javaparser-core/3.2.5/javaparser-core-3.2.5.jar at spark://172.17.0.10:35101/jars/javap

...

18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/hadoop-yarn-server-common-2.7.3.jar at spark://172.17.0.10:35101/jars/hadoop-yarn-server-common-2.7.3.jar with timestamp 1538071073780
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/hadoop-yarn-server-web-proxy-2.7.3.jar at spark://172.17.0.10:35101/jars/hadoop-yarn-server-web-proxy-2.7.3.jar with timestamp 1538071073782
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/hive-beeline-1.2.1.spark2.jar at spark://172.17.0.10:35101/jars/hive-beeline-1.2.1.spark2.jar with timestamp 1538071073786
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/hive-cli-1.2.1.spark2.jar at spark://172.17.0.10:35101/jars/hive-cli-1.2.1.spark2.jar with timestamp 1538071073787
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/hive-exec-1.2.1.spark2.jar at spark://172.17.0.10:35101/jars/hive-exec-1.2.1.spark2.jar with timestamp 1538071073789
18/09/27 17:57:53 INFO SparkC

18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/jline-2.12.1.jar at spark://172.17.0.10:35101/jars/jline-2.12.1.jar with timestamp 1538071073854
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/joda-time-2.9.3.jar at spark://172.17.0.10:35101/jars/joda-time-2.9.3.jar with timestamp 1538071073859
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/jodd-core-3.5.2.jar at spark://172.17.0.10:35101/jars/jodd-core-3.5.2.jar with timestamp 1538071073862
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/jpam-1.1.jar at spark://172.17.0.10:35101/jars/jpam-1.1.jar with timestamp 1538071073865
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/json4s-ast_2.11-3.2.11.jar at spark://172.17.0.10:35101/jars/json4s-ast_2.11-3.2.11.jar with timestamp 1538071073867
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/json4s-core_2.11-3.2.11.jar at spark://172.17.0.10:35101/jars/json4s-core_2.11-3.2.

18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/scala-compiler-2.11.8.jar at spark://172.17.0.10:35101/jars/scala-compiler-2.11.8.jar with timestamp 1538071073937
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/scala-library-2.11.8.jar at spark://172.17.0.10:35101/jars/scala-library-2.11.8.jar with timestamp 1538071073939
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/scala-parser-combinators_2.11-1.0.4.jar at spark://172.17.0.10:35101/jars/scala-parser-combinators_2.11-1.0.4.jar with timestamp 1538071073941
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/scala-reflect-2.11.8.jar at spark://172.17.0.10:35101/jars/scala-reflect-2.11.8.jar with timestamp 1538071073942
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/jars/scala-xml_2.11-1.0.2.jar at spark://172.17.0.10:35101/jars/scala-xml_2.11-1.0.2.jar with timestamp 1538071073949
18/09/27 17:57:53 INFO SparkContext: Added JAR file:/opt/spark/

18/09/27 17:57:54 INFO SparkContext: Added JAR file:/opt/spark/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.21/slf4j-api-1.7.21.jar at spark://172.17.0.10:35101/jars/slf4j-api-1.7.21.jar with timestamp 1538071074150
18/09/27 17:57:54 INFO SparkContext: Added JAR file:/opt/spark/.cache/coursier/v1/https/repo1.maven.org/maven2/org/isarnproject/isarn-sketches_2.11/0.1.2/isarn-sketches_2.11-0.1.2.jar at spark://172.17.0.10:35101/jars/isarn-sketches_2.11-0.1.2.jar with timestamp 1538071074157
18/09/27 17:57:54 INFO SparkContext: Added JAR file:/opt/spark/.cache/coursier/v1/https/repo1.maven.org/maven2/org/isarnproject/isarn-algebra-api_2.11/0.0.3/isarn-algebra-api_2.11-0.0.3.jar at spark://172.17.0.10:35101/jars/isarn-algebra-api_2.11-0.0.3.jar with timestamp 1538071074158
18/09/27 17:57:54 INFO SparkContext: Added JAR file:/opt/spark/.cache/coursier/v1/https/repo1.maven.org/maven2/org/isarnproject/isarn-sketches-spark_2.11/0.3.1-topk-1-sp2.2-py2.7/isarn-sketches-

import org.apache.spark.sql._

spark: SparkSession = org.apache.spark.sql.SparkSession@54bc942

In [5]:
import spark.sqlContext.implicits._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.streaming.Trigger
import org.isarnproject.sketches._, org.isarnproject.sketches.udaf._, org.apache.spark.isarnproject.sketches.udt._

import spark.sqlContext.implicits._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._

import org.apache.spark.sql.expressions._

import org.apache.spark.sql.streaming.Trigger

import org.isarnproject.sketches._, org.isarnproject.sketches.udaf._, org.apache.spark.isarnproject.sketches.udt._

In [6]:
val appender = org.apache.log4j.Logger.getRootLogger().getAppender("console").asInstanceOf[org.apache.log4j.ConsoleAppender]
appender.setThreshold(org.apache.log4j.Level.WARN)

appender: org.apache.log4j.ConsoleAppender = org.apache.log4j.ConsoleAppender@a9effc9

In [7]:
spark.udf.register("wordcount", (text: String)=>text.split(" ").filter(_.length > 0).length)
val sketchCDF = tdigestUDAF[Double].delta(0.2).maxDiscrete(25)
spark.udf.register("p50", (c:Any)=>c.asInstanceOf[TDigestSQL].tdigest.cdfInverse(0.5))
spark.udf.register("p90", (c:Any)=>c.asInstanceOf[TDigestSQL].tdigest.cdfInverse(0.9))
spark.udf.register("p99", (c:Any)=>c.asInstanceOf[TDigestSQL].tdigest.cdfInverse(0.99))

res6_0: UserDefinedFunction = UserDefinedFunction(
  <function1>,
  IntegerType,
  Some(List(StringType))
)
sketchCDF: TDigestUDAF[Double] = TDigestUDAF(0.2, 25)
res6_2: UserDefinedFunction = UserDefinedFunction(<function1>, DoubleType, None)
res6_3: UserDefinedFunction = UserDefinedFunction(<function1>, DoubleType, None)
res6_4: UserDefinedFunction = UserDefinedFunction(<function1>, DoubleType, None)

In [8]:
val df = spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka:9092")
  .option("subscribe", "social-firehose")
  .load()
val values = df.select(($"value").cast(StringType))
val structure = StructType(Seq("text","user_id","update_id").map{f=>StructField(f, StringType, true)})
val records = values.select(from_json($"value", structure).alias("json"))
    .select($"json.user_id", $"json.text")
    .select($"user_id",
            $"text",
            callUDF("wordcount", $"text").alias("wordcount"))

df: DataFrame = [key: binary, value: binary ... 5 more fields]
values: DataFrame = [value: string]
structure: StructType = StructType(
  StructField("text", StringType, true, {}),
  StructField("user_id", StringType, true, {}),
  StructField("update_id", StringType, true, {})
)
records: DataFrame = [user_id: string, text: string ... 1 more field]

In [9]:
val t = records.groupBy($"user_id")
    .agg(avg($"wordcount").alias("avg"))
    .orderBy($"avg".desc)
val query = t.writeStream
  .trigger(Trigger.ProcessingTime("15 seconds"))
  .outputMode("complete")
  .format("console")
  .start()
query.awaitTermination(50 * 1000)
Thread.sleep(3 * 1000)
query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+---+
|user_id|avg|
+-------+---+
+-------+---+

-------------------------------------------
Batch: 1
-------------------------------------------
+----------+----+
|   user_id| avg|
+----------+----+
|3384408632|40.0|
|7076610942|19.0|
|1656582676|18.0|
|0253301460|16.0|
|1853980848|15.0|
|9680421532|14.0|
|3237646542|14.0|
|2741764075|12.0|
|0072980186|11.0|
|4150606001|10.0|
|8826969164| 9.0|
|6969222767| 8.0|
+----------+----+

-------------------------------------------
Batch: 2
-------------------------------------------
+----------+----+
|   user_id| avg|
+----------+----+
|3384408632|40.0|
|8963165417|37.0|
|3556776499|32.0|
|0368331941|24.0|
|1996606479|21.0|
|0556078341|20.0|
|7929052071|20.0|
|7076610942|19.0|
|5591035376|19.0|
|1656582676|18.0|
|3144537041|16.0|
|0253301460|16.0|
|1853980848|15.0|
|7253011876|15.0|
|7011988820|15.0|
|9680421532|14.0|
|3237646542|14.0|
|27

t: Dataset[Row] = [user_id: string, avg: double]
query: streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3790764f
res8_2: Boolean = false

In [10]:
val t = records.withColumn("time", current_timestamp()).groupBy(window($"time", "30 seconds", "10 seconds"))
    .agg(sketchCDF($"wordcount").alias("CDF"))
    .select(callUDF("p50", $"CDF").alias("p50"),
            callUDF("p90", $"CDF").alias("p90"))
val query = t.writeStream
  .trigger(Trigger.ProcessingTime("20 seconds"))
  .outputMode("complete")
  .format("console")
  .start()
query.awaitTermination(65 * 1000)
Thread.sleep(3 * 1000)
query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+---+---+
|p50|p90|
+---+---+
+---+---+

-------------------------------------------
Batch: 1
-------------------------------------------
+----+------------------+
| p50|               p90|
+----+------------------+
|13.5|27.733333333333334|
|13.5|27.733333333333334|
|13.5|27.733333333333334|
+----+------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+------------------+
|  p50|               p90|
+-----+------------------+
| 13.5|27.733333333333334|
|15.75|              29.0|
|15.75|              29.0|
| 15.6|              29.0|
| 13.5|27.733333333333334|
+-----+------------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+-----+------------------+
|  p50|               p90|
+-----+------------------+
| 13.5|27.733333333333334|
|15.75|              29.0|
| 19.0|27

t: DataFrame = [p50: double, p90: double]
query: streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6d7ac677
res9_2: Boolean = false

In [11]:
val tka = new org.isarnproject.sketches.udaf.TopKAggregator[(String, String)](_._2).toColumn
val t = records
  .select($"user_id",
          explode(split($"text", " ")).alias("word"))
  .as[(String, String)]
  .filter(_._2(0)=='#')
  .groupByKey { case (id, _) => id }.agg(tka)
  .map { case (id, tk) => (id, tk.toVector.toString)}
val query = t.writeStream
  .trigger(Trigger.ProcessingTime("20 seconds"))
  .option("truncate", false)
  .outputMode("complete")
  .format("console")
  .start()
query.awaitTermination(65 * 1000)
Thread.sleep(3 * 1000)
query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+---+---+
|_1 |_2 |
+---+---+
+---+---+

-------------------------------------------
Batch: 1
-------------------------------------------
+----------+--------------------------------------------------------------------------------------------------------------------+
|_1        |_2                                                                                                                  |
+----------+--------------------------------------------------------------------------------------------------------------------+
|0757325932|Vector((#Marianne,1), (#AAP,1))                                                                                     |
|6927404733|Vector((#Marianne's,1), (#20+days,1), (#Crawford,1), (#allhours,1), (#Mix,1))                                       |
|9431618928|Vector((#Lyme.,1), (#onlyfiveminutes,1))                                                               

tka: TypedColumn[(String, String), Array[(String, Int)]] = topkaggregator()
t: Dataset[(String, String)] = [_1: string, _2: string]
query: streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@1604614c
res10_3: Boolean = false

In [12]:
object windowing {
    import java.sql.Timestamp
    import java.time.Instant
    def windowBy[R](f:R=>Timestamp, width: Int) = {
        val w = width.toLong * 1000L
        (row: R) => {
            val tsCur = f(row)
            val msCur = tsCur.getTime()
            val msLB = (msCur / w) * w
            val instLB = Instant.ofEpochMilli(msLB)
            val instUB = Instant.ofEpochMilli(msLB+w)
            (Timestamp.from(instLB), Timestamp.from(instUB))
        }
    }
}

defined object windowing

In [13]:
val windowBy60 = windowing.windowBy[(java.sql.Timestamp, String)](_._1, 60)
val tka = new org.isarnproject.sketches.udaf.TopKAggregator[(java.sql.Timestamp, String)](_._2).toColumn
val t = records
  .select(current_timestamp().alias("time"),
          explode(split($"text", " ")).alias("word"))
  .as[(java.sql.Timestamp, String)]
  .filter(_._2(0)=='#')
  .groupByKey(windowBy60).agg(tka)
  .map { case (tw, tk) => (tw._2, tk.toVector.toString)}
val query = t.writeStream
  .trigger(Trigger.ProcessingTime("20 seconds"))
  .option("truncate", false)
  .outputMode("complete")
  .format("console")
  .start()
query.awaitTermination(125 * 1000)
Thread.sleep(3 * 1000)
query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+---+---+
|_1 |_2 |
+---+---+
+---+---+

-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+----------------------------------------------------------------------------+
|_1                 |_2                                                                          |
+-------------------+----------------------------------------------------------------------------+
|2018-09-27 18:02:00|Vector((#PETE,2), (#Nutiva,1), (#Fairfax,1), (#H.Halton,1), (#Afewmonths,1))|
+-------------------+----------------------------------------------------------------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-------------------+----------------------------------------------------------------------------+
|_1                 |_2                                                  

18/09/27 18:03:22 WARN JobProgressListener: Task start for unknown stage 115
18/09/27 18:03:22 WARN TaskSetManager: Lost task 121.0 in stage 115.0 (TID 9637, localhost, executor driver): TaskKilled (stage cancelled)


windowBy60: (java.sql.Timestamp, String) => (java.sql.Timestamp, java.sql.Timestamp) = <function1>
tka: TypedColumn[(java.sql.Timestamp, String), Array[(String, Int)]] = topkaggregator()
t: Dataset[(java.sql.Timestamp, String)] = [_1: timestamp, _2: string]
query: streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@34fa0ca0
res12_4: Boolean = false